In [7]:
import os, json, shutil

# 원본 이미지 폴더
image_root = r"C:\Users\user\.cache\kagglehub\datasets\dansbecker\food-101\versions\1\food-101\food-101\images"

# json 경로
json_path = r"C:\Users\user\.cache\kagglehub\datasets\dansbecker\food-101\versions\1\food-101\food-101\meta\test.json"  # 업로드한 test.json 파일 경로
output_dir = "data/val"  # validation 폴더로 사용 (ImageFolder 포맷)

# JSON 불러오기
with open(json_path, 'r') as f:
    data = json.load(f)

# 클래스별로 복사
for label, paths in data.items():
    class_dir = os.path.join(output_dir, label)
    os.makedirs(class_dir, exist_ok=True)

    for rel_path in paths:
        full_src = os.path.join(image_root, rel_path + ".jpg")
        full_dst = os.path.join(class_dir, os.path.basename(rel_path) + ".jpg")
        if os.path.exists(full_src):
            shutil.copy(full_src, full_dst)

print("✅ test.json 기준 val 폴더 생성 완료!")


✅ test.json 기준 val 폴더 생성 완료!


In [8]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

val_dataset = datasets.ImageFolder('data/val', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32)


In [ ]:
import os, json, shutil

# 원본 이미지 위치
image_root = r"C:\Users\user\.cache\kagglehub\datasets\dansbecker\food-101\versions\1\food-101\food-101\images"

# meta 파일 경로
meta_root = r"C:\Users\user\.cache\kagglehub\datasets\dansbecker\food-101\versions\1\food-101\food-101\meta"
train_json = os.path.join(meta_root, "train.json")
test_json = os.path.join(meta_root, "test.json")

def make_imagefolder_structure(json_path, out_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)

    for label, paths in data.items():
        class_dir = os.path.join(out_dir, label)
        os.makedirs(class_dir, exist_ok=True)

        for rel_path in paths:
            src = os.path.join(image_root, rel_path + ".jpg")
            dst = os.path.join(class_dir, os.path.basename(rel_path) + ".jpg")
            if os.path.exists(src):
                shutil.copy(src, dst)

# 실행
make_imagefolder_structure(train_json, "data/train")
make_imagefolder_structure(test_json, "data/val")

print("✅ train/val ImageFolder 구성 완료!")


In [ ]:
import torch, os, json
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn, optim
from model_def import FoodCNN  # model_def.py는 FastAPI에서도 재사용할 구조

# 1. 데이터
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = datasets.ImageFolder("data/train", transform=transform)
val_dataset = datasets.ImageFolder("data/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# 2. 모델
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FoodCNN(num_classes=len(train_dataset.classes)).to(device)

# 3. 손실함수 & 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. 학습
for epoch in range(5):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        preds = model(x)
        loss = criterion(preds, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"[{epoch+1}] Train Loss: {total_loss:.4f}")

# 5. 저장
os.makedirs("app/model", exist_ok=True)
torch.save(model.state_dict(), "app/model/model.pth")

# 6. 클래스 저장
with open("app/model/classes.json", "w") as f:
    json.dump(train_dataset.classes, f)

print("✅ 모델 저장 완료 (model.pth + classes.json)")
